<a href="https://colab.research.google.com/github/VlasovArs/Recommendation_systems/blob/main/5_HW_by_Demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from surprise import Dataset
from surprise import Reader


In [ ]:
from surprise import SVD

In [ ]:
from surprise import KNNBaseline
from surprise.model_selection import cross_validate

## Задание:
```data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.30, random_state=999)```
1. Для параметров i = [1,5,9,12,17,22] найдите лучший SVD (random_state=999) алгоритм по метрике MAP(n=7)

2. Сделайте прогноз по всем пользователям, который равен среднему из KNN (k=15, c корреляцией Пирсона и item-based подходом) и SVD (n = лучший из пункта 1).
Сравните по метрике MAP(n=7) полученные предсказания с kNN и SVD.

### Задача 1

In [ ]:
from surprise.model_selection import train_test_split

In [ ]:
def Precision_at_n(df_ratings, n=3, threshold=5):
    rel_n = set(df_ratings[df_ratings.real_rating>=threshold]['itemID'])
    t=df_ratings.sort_values(by='rating', ascending=False)[0:n]
    ret_n=set(t['itemID'])
    return len(rel_n & ret_n)/n

In [ ]:
def Avg_Precision_at_n(df_ratings, n=10,threshold=5):
    avg_p=0
    rel_n = set(df_ratings[df_ratings.real_rating>=threshold]['itemID'])

    for i in range(1,n+1):
        t=df_ratings.sort_values(by='rating', ascending=False)[0:i]
        ret_n=set(t['itemID'])
        t2=df_ratings.sort_values(by='rating', ascending=False)[i-1:i]
        ret_n2=set(t2['itemID'])
        #print(ret_n2,ret_n)
        if len((ret_n2 & rel_n))>0:
            avg_p=avg_p+len(rel_n & ret_n)/n
    return avg_p/n

In [ ]:
def MAP_at_n(df_ratings_all, n=10,threshold=5):
    users=df_ratings_all['userID'].unique()
    map_at_n=0
    for i in users:
        df_ratings=df_ratings_all[df_ratings_all.userID==i]
        map_at_n=map_at_n+Avg_Precision_at_n(df_ratings,n=n,threshold=threshold)
    return map_at_n/len(users)

In [ ]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.30, random_state=999)

In [ ]:
for k in [1,5,9,12,17,22]:
    algo = SVD(n_factors=k,random_state=999, verbose=False) #отключим вывод логирования
    predictions = algo.fit(trainset).test(testset)
    df_ratings=pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
    for i in predictions:
        df_ratings = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)
    print(k,MAP_at_n(df_ratings,7))

1 0.11792585538987567
5 0.11985197048066282
9 0.11985197048066278
12 0.11889973380656582
17 0.12229748739368461
22 0.11768779622135146


In [ ]:
#17

### Задача 2

In [ ]:
from surprise import KNNBaseline

In [ ]:
knn=KNNBaseline(k=15,sim_options = {'name': 'pearson_baseline','user_based': False}, verbose=False)
svd = SVD(n_factors=17,random_state=999, verbose=False)

In [ ]:
predictions = knn.fit(trainset).test(testset)
df_ratings=pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions:
    df_ratings = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

In [ ]:
df_ratings.head()

,userID,itemID,rating,real_rating
0,753,357,3.601156,4.0
1,542,585,2.541421,2.0
2,5,364,2.906434,1.0
3,887,385,4.244959,4.0
4,383,480,4.868237,5.0


In [ ]:
predictions = svd.fit(trainset).test(testset)
df_ratings2=pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions:
    df_ratings2 = df_ratings2.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

In [ ]:
df_ratings2.head()

,userID,itemID,rating,real_rating
0,753,357,3.543399,4.0
1,542,585,2.627620,2.0
2,5,364,2.345794,1.0
3,887,385,4.106974,4.0
4,383,480,4.550769,5.0


In [ ]:
df_knn_svd=df_ratings.merge(df_ratings2[['userID','itemID','rating']],how='left', on=['userID','itemID'])

In [ ]:
df_knn_svd['rating']=(df_knn_svd['rating_x']+ df_knn_svd['rating_y'])/2

In [ ]:
df_knn_svd.head()

,userID,itemID,rating_x,real_rating,rating_y,rating
0,753,357,3.601156,4.0,3.543399,3.572277
1,542,585,2.541421,2.0,2.627620,2.584520
2,5,364,2.906434,1.0,2.345794,2.626114
3,887,385,4.244959,4.0,4.106974,4.175967
4,383,480,4.868237,5.0,4.550769,4.709503


In [ ]:
print(MAP_at_n(df_ratings,7),MAP_at_n(df_ratings2,7),MAP_at_n(df_knn_svd,7))

0.12677732811045903 0.12229748739368461 0.12775120652714916
